In [ ]:
!pip install -q gradio sentence-transformers faiss-cpu requests beautifulsoup4

Exception ignored in: <function _xla_gc_callback at 0x7f9d5e731440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/jax/_src/lib/__init__.py", line 127, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


In [ ]:
import gradio as gr
import requests, re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [ ]:
SOURCES = {
    "Machine Learning": [
        ("Goodfellow ML", "https://www.deeplearningbook.org/contents/ml.html"),
        ("Andrew Ng ML (Notes)", "https://cs229.stanford.edu/"),
        ("MIT Intro to ML", "https://introtoml.mit.edu/"),
        ("Google ML Crash Course", "https://developers.google.com/machine-learning/crash-course")
    ],

    "Computer Vision": [
        ("CS231n Stanford", "http://cs231n.stanford.edu/"),
        ("PyImageSearch CV", "https://www.pyimagesearch.com/start-here/"),
        ("OpenCV Tutorials", "https://docs.opencv.org/4.x/d9/df8/tutorial_root.html"),
        ("Learn PyTorch CV", "https://www.learnpytorch.io/")
    ],

    "NLP": [
        ("CS224n Stanford", "https://cs224n.stanford.edu/"),
        ("Jurafsky NLP Book", "https://web.stanford.edu/~jurafsky/slp3/"),
        ("HuggingFace NLP Course", "https://huggingface.co/learn/nlp-course"),
        ("NLTK Book", "https://www.nltk.org/book/")
    ]
}

In [ ]:
# ---------------- ЗАГРУЗКА ТЕКСТОВ (УЛУЧШЕННАЯ) ----------------
def load_docs():
    docs = []

    for topic, items in SOURCES.items():
        for name, url in items:
            try:
                r = requests.get(
                    url,
                    timeout=15,
                    headers={"User-Agent": "Mozilla/5.0"}
                )
                r.raise_for_status()

                soup = BeautifulSoup(r.text, "html.parser")

                # Убираем мусорные теги
                for tag in soup(["script", "style", "nav", "header", "footer", "aside"]):
                    tag.decompose()

                text = soup.get_text(separator=" ")
                text = re.sub(r"\s+", " ", text).strip()

                # Ограничение размера (важно для скорости и памяти)
                text = text[:4000]

            except Exception as e:
                # fallback, чтобы RAG не ломался
                text = f"Educational material about {name} ({topic})"

            docs.append({
                "topic": topic,
                "name": name,
                "url": url,
                "text": text
            })

    return docs


docs = load_docs()

In [ ]:
# ---------------- ЧАНКИНГ ----------------
def chunk_text(
    text: str,
    chunk_size: int = 600,
    overlap: int = 120
):
    """
    Делит текст на чанки с перекрытием.
    chunk_size и overlap — в символах.
    """
    text = re.sub(r"\s+", " ", text).strip()
    chunks = []

    start = 0
    text_len = len(text)

    while start < text_len:
        end = min(start + chunk_size, text_len)
        chunk = text[start:end].strip()

        # Фильтр слишком коротких чанков
        if len(chunk) >= 200:
            chunks.append(chunk)

        # шаг назад для overlap
        start = end - overlap
        if start < 0:
            start = 0

        if end == text_len:
            break

    return chunks


# применяем ко всем документам
chunks = []
for d in docs:
    text_chunks = chunk_text(d["text"])
    for ch in text_chunks:
        chunks.append({
            "topic": d["topic"],
            "name": d["name"],
            "url": d["url"],
            "text": ch
        })

print(f"✅ Создано чанков: {len(chunks)}")

In [ ]:
# ---------------- МОДЕЛЬ + FAISS ----------------

# 1. Загружаем модель эмбеддингов (лёгкая и стабильная)
model = SentenceTransformer("all-MiniLM-L6-v2")

# 2. Кодируем все чанки
texts = [c["text"] for c in chunks]
embeddings = model.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

# 3. FAISS индекс (cosine similarity через L2 + normalization)
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype("float32"))

print(f"✅ FAISS индекс создан | чанков: {len(chunks)} | dim: {dimension}")


In [ ]:
# ---------------- RAG ПОИСК ----------------
def rag_search(query: str, k: int = 3):
    """
    Возвращает top-k наиболее релевантных чанков
    """
    q_emb = model.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )

    distances, indices = index.search(q_emb.astype("float32"), k)

    results = []
    for rank, idx in enumerate(indices[0]):
        if idx < len(chunks):
            results.append({
                "rank": rank + 1,
                "score": float(distances[0][rank]),
                "topic": chunks[idx]["topic"],
                "name": chunks[idx]["name"],
                "url": chunks[idx]["url"],
                "text": chunks[idx]["text"]
            })

    return results

In [ ]:
# ---------------- ЛОГИКА UI ----------------
def get_lesson(topic, level, query):
    results = rag_search(f"{query} {topic} {level}")

    # Заголовок
    out = f"## 🧠 {query}\n"
    out += f"**Направление:** {topic}  \n"
    out += f"**Уровень:** {level}\n\n"
    out += "---\n\n"

    # Блок с источниками
    out += "### 📚 Релевантные фрагменты из источников\n\n"

    for r in results:
        out += f"**{r['rank']}. {r['name']}**  \n"
        out += f"*Relevance score:* `{r['score']:.3f}`  \n\n"
        out += f"{r['text'][:400]}...\n\n"
        out += f"🔗 [{r['url']}]({r['url']})\n\n"
        out += "---\n\n"

    # Контрольный вопрос
    out += "### ❓ Контрольный вопрос\n"
    out += (
        f"Что такое **{query.lower()}** в контексте **{topic}**, "
        f"и какие методы применяются на уровне **{level}**?"
    )

    return out

In [ ]:
# ================== RAG + GRADIO (SAFE MODE) ==================

import gradio as gr
import requests, re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# ---------- 1. ИСТОЧНИКИ ----------
SOURCES = {
    "Machine Learning": [
        ("Goodfellow ML", "https://www.deeplearningbook.org/contents/ml.html"),
        ("Andrew Ng ML", "https://www.coursera.org/learn/machine-learning")
    ],
    "Computer Vision": [
        ("CS231n", "http://cs231n.stanford.edu/")
    ],
    "NLP": [
        ("CS224n", "https://cs224n.stanford.edu/")
    ]
}

# ---------- 2. ЗАГРУЗКА ТЕКСТОВ ----------
def load_docs():
    docs = []
    for topic, items in SOURCES.items():
        for name, url in items:
            try:
                r = requests.get(url, timeout=10, headers={"User-Agent": "Mozilla/5.0"})
                soup = BeautifulSoup(r.text, "html.parser")
                text = soup.get_text(" ")
                text = re.sub(r"\s+", " ", text)[:4000]
            except:
                text = ""
            docs.append({
                "topic": topic,
                "name": name,
                "url": url,
                "text": text
            })
    return docs

docs = load_docs()

# ---------- 3. ЧАНКИ ----------
chunks = []
for d in docs:
    for i in range(4):
        chunk_text = d["text"][i*500:(i+1)*500]
        if len(chunk_text) > 100:
            chunks.append({
                "topic": d["topic"],
                "name": d["name"],
                "url": d["url"],
                "text": chunk_text
            })

# ---------- 4. FAISS ----------
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode([c["text"] for c in chunks])

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings.astype("float32"))

def rag_search(query, k=3):
    q = model.encode([query])
    _, idx = index.search(q.astype("float32"), k)
    return [chunks[i] for i in idx[0]]

# ---------- 5. ЛОГИКА ВЫВОДА (БЕЗ LLM) ----------
def get_rag_answer(topic, level, query):
    results = rag_search(f"{query} {topic}")

    if not results:
        return "❌ Ничего не найдено."

    out = f"## 📚 RAG-результаты\n"
    out += f"**Тема:** {query}\n"
    out += f"**Направление:** {topic}\n"
    out += f"**Уровень:** {level}\n\n"

    for r in results:
        out += f"### 🔹 {r['name']}\n"
        out += f"{r['text'][:300]}...\n\n"
        out += f"🔗 {r['url']}\n\n---\n"

    return out

# ---------- 6. GRADIO UI ----------
with gr.Blocks() as demo:
    gr.Markdown("# 🧠 RAG-репетитор (без LLM, с цитатами)")
    gr.Markdown("Стабильный режим: поиск по учебным источникам.")

    topic = gr.Radio(
        ["Machine Learning", "Computer Vision", "NLP"],
        label="📚 Направление",
        value="Machine Learning"
    )

    level = gr.Radio(
        ["Beginner", "Middle", "Senior"],
        label="🎓 Уровень",
        value="Beginner"
    )

    query = gr.Textbox(
        label="Тема",
        value="Overfitting"
    )

    btn = gr.Button("🔍 Найти в источниках")
    output = gr.Markdown()

    btn.click(
        fn=get_rag_answer,
        inputs=[topic, level, query],
        outputs=output
    )

demo.launch(debug=True)

# =============================================================

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e56851397d38837800.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
